In [ ]:
# MNIST save model p4 (Sequential)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import torch
import torch.utils.data as data
import json
from PIL import Image
import torchvision.transforms.v2 as tfs
import torchvision
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

# GPU setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


class RavelTransform(nn.Module):
    def forward(self,item):
        return item.ravel()


# Options to buld models
# option 1
model_0 = nn.Sequential(
    nn.Linear(28 *28, 32),
    nn.ReLU(),
    nn.Linear(32,10)
)

# option2
model_1 = nn.Sequential()
model_1.add_module('layer_1', nn.Linear(28 *28, 32))
model_1.add_module('relu',nn.ReLU())
model_1.add_module('layer_2', nn.Linear(32, 10))


# option3
class DigitNN(nn.Module):
    def __init__(self, input_dim, num_hid, out_dim):
        super().__init__()
        self.net = nn.Sequential(
                    nn.Linear(28 *28, 32),
                    nn.ReLU(),
                    nn.Linear(32,10)
                )
    def forward(self,x):
        x = self.net(x)
        return x
model_2 = DigitNN(28 * 28, 128, 10) 

# option4
block = nn.Sequential(
    nn.Linear(32, 32),
    nn.LeakyReLU(),
    nn.Linear(32,16),
    nn.LeakyReLU()
    )
model_3 = nn.Sequential()
model_3.add_module('layer_1', nn.Linear(28 *28, 32))
model_3.add_module('relu',nn.ReLU())
model_3.add_module('block', block)
model_3.add_module('layer_2', nn.Linear(16, 10))

# option5 ModuleList (Нужен для регистрации всех слоев в моделе)
class DigitNN2(nn.Module):
    def __init__(self, input_dim = 28*28, out_dim = 10, n_layers=3):
        super().__init__()
        self.layers = nn.ModuleList(
            [ nn.Linear( input_dim // n, input_dim // (n+1)) for n in range(1, n_layers+1) ])

        self.net = nn.Sequential(
                    nn.Linear(196, 32),
                    nn.ReLU(),
                    nn.Linear(32,10)
                )
    def forward(self, x):
        for layer in self.layers:
            x = layer(x).tanh()

        x = self.net(x)
        return x
model_4 = DigitNN2() 

# option 6 ModuleList+Sequential
class DigitNN3(nn.Module):
    def __init__(self, input_dim = 28*28, out_dim = 10, n_layers=3):
        super().__init__()
        self.layers = [nn.Linear( input_dim // n, input_dim // (n+1)) for n in range(1, n_layers+1)]
        self.net0 = nn.Sequential(*self.layers)

        self.net = nn.Sequential(
                    nn.Linear(196, 32),
                    nn.ReLU(),
                    nn.Linear(32,10)
                )
    def forward(self, x):
        for layer in self.net0:
            x = layer(x).tanh()
        x = self.net(x)
        return x
model_5 = DigitNN3() 
model = model_5

# option 7 ModuleList with names
class DigitNN4(nn.Module):
    def __init__(self, input_dim = 28*28, out_dim = 10, n_layers=3):
        super().__init__()
        self.layers = nn.ModuleList()
        for n in range(1, n_layers+1):
            self.layers.add_module(f"layer_{n}", nn.Linear(input_dim // n, input_dim // (n+1) ))

        self.net = nn.Sequential(
                    nn.Linear(196, 32),
                    nn.ReLU(),
                    nn.Linear(32,10)
                )
    def forward(self, x):
        for layer in self.layers:
            x = layer(x).tanh()
        x = self.net(x)
        return x
model_6 = DigitNN4() 
model = model_6

model.to(device)
print(model.layers.layer_3)

# Option 8 ModuleDict
class DigitNN5(nn.Module):
    def __init__(self, input_dim = 28*28, out_dim = 10, n_layers=3, act_type = None):
        super().__init__()
        self.layers = nn.ModuleList()
        self.act_type = act_type
        for n in range(1, n_layers+1):
            self.layers.add_module(f"layer_{n}", nn.Linear(input_dim // n, input_dim // (n+1) ))

        self.net = nn.Sequential(
                    nn.Linear(196, 32),
                    nn.ReLU(),
                    nn.Linear(32,10)
                )
        self.act_lst = nn.ModuleDict({
            'relu': nn.ReLU(),
            'lk_relu':nn.LeakyReLU()
        })
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
            # if (Nullcheck) and (key in dict)
            if self.act_type and self.act_type in self.act_lst:
                x = self.act_lst[self.act_type](x)
        x = self.net(x)
        return x
model_7 = DigitNN5() 
model = model_7

model.to(device)
print(model)


# # Setting trnsform
# transforms = tfs.Compose([tfs.ToImage(),
#                           tfs.Grayscale(), 
#                           tfs.ToDtype(torch.float32, scale=True), # converting to float
#                           RavelTransform() # or tfs.Lambda(lambda _im: _im.ravel()),
#                           ])


# dataset_mnist = torchvision.datasets.MNIST("datasets\mnist", download=True, train=True, transform=transforms)
# d_train, d_val = data.random_split(dataset_mnist , [0.7, 0.3])
# train_data = data.DataLoader(d_train, batch_size=32, shuffle=True)
# train_data_val = data.DataLoader(d_val, batch_size=32, shuffle=False)

# optimizer = optim.Adam(params=model.parameters(), lr=0.01)
# loss_function = nn.CrossEntropyLoss()

# # Lists to store metrics for plotting
# train_losses = []
# val_losses = []

# ep = 3
# for _e in range(ep):
#     model.train()
#     loss_mean = 0
#     loss_count = 0
    
#     train_data_tqdm = tqdm(train_data, leave=True)
#     for x_train_img, y_target in train_data_tqdm:
#         # Move data to GPU
#         x_train_img, y_target = x_train_img.to(device), y_target.to(device)
        
#         predict = model(x_train_img)
#         loss = loss_function(predict, y_target)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         loss_count +=1
#         loss_mean = 1/loss_count * loss.item() + (1 - 1/loss_count) * loss_mean
#         train_data_tqdm.set_description(f"Epoch [{_e+1}/{ep}], loss_mean={loss_mean:.3f}")

#     train_losses.append(loss_mean)

#     # Validataion
#     model.eval()
#     Q_val = 0
#     count_val = 0

#     with torch.no_grad():  # No gradients needed for validation
#         for x_val_img , y_target_val in train_data_val:
#             # Move data to GPU
#             x_val_img, y_target_val = x_val_img.to(device), y_target_val.to(device)
            
#             p = model(x_val_img)
#             loss = loss_function(p,y_target_val)
#             Q_val+= loss.item()
#             count_val +=1
#     Q_val /= count_val
#     val_losses.append(Q_val)

#     print(f"loss_mean = {loss_mean:.3f}, Q_val = {Q_val:.3f}")


# # Test
# d_test = ImageFolder("datasets/MNIST/test", transform=transforms)
# test_data = data.DataLoader(d_test , batch_size=500, shuffle=False)
# Q = 0

# model.eval()

# test_tqdm = tqdm(test_data, leave=True)
# with torch.no_grad():  # No gradients needed for testing
#     for x_test, y_test in test_tqdm:
#         # Move data to GPU
#         x_test, y_test = x_test.to(device), y_test.to(device)
        
#         p = model(x_test) # 500, 10
#         p = torch.argmax(p, dim=1) # 500,1
#         Q += torch.sum(p == y_test).item()

# Q /= len(d_test)
# print(f"Test Accuracy: {Q:.4f}")


# # Plot training and validation losses
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, ep+1), train_losses, marker='o', label='Training Loss', linewidth=2)
# plt.plot(range(1, ep+1), val_losses, marker='s', label='Validation Loss', linewidth=2)
# plt.xlabel('Epoch', fontsize=12)
# plt.ylabel('Loss', fontsize=12)
# plt.title('Training and Validation Loss Over Epochs', fontsize=14)
# plt.legend(fontsize=11)
# plt.grid(True, alpha=0.3)
# plt.tight_layout()
# plt.savefig('training_validation_loss.png', dpi=150)
# plt.show()

# print(f"\nFinal Results:")
# print(f"Training Loss: {train_losses[-1]:.4f}")
# print(f"Validation Loss: {val_losses[-1]:.4f}")
# print(f"Test Accuracy: {Q:.4f}")

Using device: cuda
Linear(in_features=261, out_features=196, bias=True)


In [2]:
# 1
import torch
import torch.nn as nn

batch_size=8
x = torch.rand(batch_size, 5) # тензор x в программе не менять

# здесь продолжайте программу

model = nn.Sequential(
    nn.Linear(5, 16, bias=False),
    nn.ReLU(),
    nn.BatchNorm1d(16),
    nn.Linear(16,3)
)

model.eval()
predict = model(x)

In [3]:
# 2
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim

# пропишите модель нейронной сети
model = nn.Sequential(
    nn.Linear(3, 1)
) 

# создание обучающей выборки
_x = torch.arange(-5, 5, 0.1)
data_y = torch.sin(2 * _x) + 0.2 * torch.cos(10 * _x) + 0.1 * _x ** 2

_x.unsqueeze_(-1)
data_x = torch.cat([_x, _x ** 2, _x ** 3], dim=1)
ds = data.TensorDataset(data_x, data_y)

epochs = 20 # число эпох обучения
batch_size = 8 # размер батча

# создать объект класса DataLoader для датасета ds с размером пакетов batch_size и перемешиванием образов выборки
train_data = data.DataLoader(ds,batch_size, shuffle=True)

# создать оптимизатор RMSprop для обучения модели с шагом обучения 0.01
optimizer = optim.RMSprop(params=model.parameters(), lr=0.01) 
# создать функцию потерь с помощью класса MSELoss
loss_func = nn.MSELoss()


# перевести модель в режим обучения
model.train()
for _e in range(epochs): # итерации по эпохам
    for x_train, y_train in train_data:
        predict = model(x_train) # вычислить прогноз модели для данных x_train
        loss = loss_func(predict, y_train.unsqueeze(-1)) # вычислить значение функции потерь

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# перевести модель в режим эксплуатации
model.eval()
# выполнить прогноз модели по всем данным выборки (d_train.data)
p = model(data_x)
# вычислить потери с помощью loss_func по всем данным выборки; значение Q сохранить в виде вещественного числа
Q = loss_func(p , data_y.unsqueeze(-1)).item()

In [6]:
# 3
import torch
import torch.nn as nn

batch_size=16
x = torch.rand(batch_size, 12) # тензор x в программе не менять

# здесь продолжайте программу
model = nn.Sequential()
model.add_module("layer1",nn.Linear(12,24, bias=True) )
model.add_module("act1",nn.Tanh() )
model.add_module("layer2", nn.Linear(24,10, bias=True) )
model.add_module("act2", nn.Tanh() )
model.add_module("layer3", nn.Linear(10,1, bias=True) )

model.eval()
predict = model(x)
print(model.layer1)

Linear(in_features=12, out_features=24, bias=True)


In [2]:
# 4
import torch
import torch.nn as nn

batch_size=12
x = torch.rand(batch_size, 64) # тензор x в программе не менять

# здесь продолжайте программу
block_bm_dp = nn.Sequential(
    nn.Linear(32,32, bias=False),
    nn.ELU(),
    nn.BatchNorm1d(32),
    nn.Dropout(0.3)
)

model = nn.Sequential()
model.add_module("input", nn.Linear(64,32,bias=True))
model.add_module("act1", nn.ReLU())
for b in range(1,4):
    model.add_module(f"block{b}",block_bm_dp )
model.add_module("output", nn.Linear(32,10, bias=True))

model.eval()
predict = model(x)


In [4]:
# 5
import torch
import torch.nn as nn

# здесь объявляйте класс модели

batch_size=12
a = torch.rand(batch_size, 7) # тензоры a, b в программе не менять
b = torch.rand(batch_size, 12)

# здесь продолжайте программу
class MyModel(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()
        self.inp_1 = nn.Sequential(
            nn.Linear(7,12,bias=True),
            nn.Tanh()
        )
        self.inp_2 = nn.Sequential(
            nn.Linear(12,12,bias=True),
            nn.Sigmoid()
        )
        self.out =  nn.Sequential(
            nn.Linear(12,32,bias=True),
            nn.ReLU(),
            nn.Linear(32,1,bias=True)
        )

    def forward(self, a, b):
        a = self.inp_1(a)
        b = self.inp_2(b)
        c = self.out(a+b)
        return c

model = MyModel()
model.eval()
predict = model(a,b)

print(model.out)

Sequential(
  (0): Linear(in_features=12, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=1, bias=True)
)


In [1]:
# 6
import torch
import torch.nn as nn

# здесь объявляйте класс модели

batch_size=28
x = torch.rand(batch_size, 32) # тензор x в программе не менять

# здесь продолжайте программу

class MyModel(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()
        self.inp = nn.Sequential(
            nn.Linear(32,64,bias=False),
            nn.LeakyReLU(),
            nn.BatchNorm1d(64)
        )

        self.out_1 = nn.Sequential(
            nn.Linear(64,10 , bias=True),
            nn.Sigmoid())
        
        self.out_2 = nn.Sequential(
            nn.Linear(64,32 , bias=True),
            nn.Tanh())
        
    def forward(self, x):
        x = self.inp(x)
        y = self.out_1(x)
        t = self.out_2(x)
        return y, t

model  = MyModel()
model.eval()
predict_y, predict_t = model(x)

In [2]:
# 7
import torch
import torch.nn as nn


class MyModel(nn.Module):
    def __init__(self, in_features, out_features, n_hidden_layers, hidden_features):
        super().__init__()
        self.layers = nn.ModuleList(
            [nn.Linear(hidden_features, hidden_features) for _ in range(n_hidden_layers)]
        )
        self.layer1 = nn.Linear(in_features, hidden_features)
        self.out = nn.Linear(hidden_features, out_features)

    def forward(self, x):
        x = self.layer1(x)
        x = torch.relu(x)
        for layer in self.layers:
            x = layer(x)
            x = torch.relu(x)
        x = self.out(x)
        return x


model = MyModel(8, 2, 5, 16)
print(model)

MyModel(
  (layers): ModuleList(
    (0-4): 5 x Linear(in_features=16, out_features=16, bias=True)
  )
  (layer1): Linear(in_features=8, out_features=16, bias=True)
  (out): Linear(in_features=16, out_features=2, bias=True)
)


In [9]:
# 8
import torch
import torch.nn as nn

# здесь объявляйте класс модели

n = int(input()) # это значение в программе не менять

batch_size = 18
x = torch.rand(batch_size, 11) # тензор x в программе не менять

# здесь продолжайте программу
class DeepNetwork(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.input = nn.Linear(11,32, bias=False)
        self.layers = nn.ModuleList(
            [nn.Linear(32, 32) for _ in range(n)]
        )
        self.output = nn.Linear(32,5, bias=True)
    
    def forward(self, x):
        x = self.input(x).relu()
        for layer in self.layers:
            x = layer(x).relu()
        y = self.output(x)
        return y
        
model = DeepNetwork(n)
model.eval()

predict = model(x)


In [ ]:
# 10
import torch
import torch.nn as nn

# здесь объявляйте класс модели

batch_size = 100
x = torch.rand(batch_size, 64) # тензор x в программе не менять

# здесь продолжайте программу
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(64,32,bias=True)
        self.blocks = nn.ModuleDict({
            'block_1': nn.Sequential(
            nn.Linear(32,32,bias=False),
            nn.ELU(),
            nn.BatchNorm1d(32)),
            'block_2' : nn.Sequential(
            nn.Linear(32,32,bias=True),
            nn.ReLU(),
            nn.Dropout(0.4)) })
        self.output = nn.Linear(32,10,bias=True)
    def forward(self, x, type_block  = 'block_1'):
        x = self.input(x).relu()
        x = self.blocks[type_block ](x)
        x = self.output(x)
        return x

model = MyModel()
model.eval()
predict = model(x, type_block='block_2' )